# HW_2

# Qianle Tang

In [43]:
import numpy as np

# Problem 1

In [44]:
class localvolDynamics:
    
    def __init__(self, S0, r, q, maxvol, localvol): 
        self.S0 = S0
        self.r = r
        self.q = q
        self.maxvol = maxvol
        self.localvol = localvol


In [45]:
hw2dynamics = localvolDynamics(S0 = 100, r = 0.06, q = 0.01, maxvol = 0.6, 
                     localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6))

# Note that hw2dynamics.localvol is a function 
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [46]:
class CallOnAmericanPut:

    def __init__(self, putexpiry, putstrike, callexpiry, callstrike):
        self.putexpiry = putexpiry
        self.putstrike = putstrike
        self.callexpiry = callexpiry
        self.callstrike = callstrike
        

In [47]:
hw2contract = CallOnAmericanPut(putexpiry=0.75, putstrike=95, callexpiry=0.25, callstrike=10)

In [48]:
class TrinomTree:
    
    def __init__(self, N):
        self.N = N
        
    # You complete the coding of this function
    def price_compound_localvol(self, contract, dynamics):
        
        deltat = 0.75 / self.N
        
        if not abs(contract.callexpiry / deltat- round(contract.callexpiry / deltat))<= 1e-12:
            raise ValueError("This value of N fails to place the call expiry in the tree.")
        
        representative_vol = dynamics.localvol(dynamics.S0, 0)
        
        deltax = max(representative_vol * np.sqrt(3 * deltat), dynamics.maxvol * np.sqrt(deltat))
        
        Sgrid = dynamics.S0*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)  
        
        put_price = np.maximum(contract.putstrike - Sgrid, 0)
        
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat:
            
            Sgrid = Sgrid[1:-1]
            
            sigma = dynamics.localvol(Sgrid, t)
            
            nu = (dynamics.r - dynamics.q) - 0.5 * sigma ** 2
            P_u = 0.5 * ((sigma ** 2 * deltat + nu ** 2 * deltat ** 2) / deltax ** 2 + nu * deltat / deltax)
            P_d = 0.5 * ((sigma ** 2 * deltat + nu ** 2 * deltat ** 2) / deltax ** 2 - nu * deltat / deltax)
            P_m = 1 - (sigma ** 2 * deltat + nu ** 2 * deltat ** 2) / deltax ** 2
            
            up_put= put_price[:-2]
            mid_put = put_price[1:-1]
            down_put = put_price[2:]

            local_value = np.maximum(contract.putstrike - Sgrid, 0) 
            
            put_price = np.maximum(local_value,
                                   np.exp(-dynamics.r * deltat) * (P_u * up_put + P_m * mid_put + P_d * down_put))

            #if abs(t - contract.callexpiry)<= 1e-12 :
            if abs(t-contract.callexpiry) <=1e-12 :   
                call_price = np.maximum(put_price - contract.callstrike, 0)
            elif t < contract.callexpiry:
                up_call = call_price[:-2]
                mid_call = call_price[1:-1]
                down_call = call_price[2:]

                call_price = np.exp(-dynamics.r * deltat) * (P_u * up_call + P_m * mid_call+ P_d * down_call)
                #no need to choose max: European style, sno way to exercise prior to expiration. 
    
        
        price_of_put = put_price[0] #write code to compute this    

        price_of_call_on_put = call_price[0] #write code to compute this
    
        return (price_of_put, price_of_call_on_put)
    

In [49]:
hw2tree = TrinomTree(N=3000)  #change if necessary to get $0.01 accuracy, in your judgment

In [50]:
(answer_part_a, answer_part_b) = hw2tree.price_compound_localvol(hw2contract,hw2dynamics)

In [52]:
print("the price of the American style put is: ",answer_part_a )
print("the price of the European style call is:",answer_part_b )

the price of the American style put is:  7.007352296510587
the price of the European style call is: 1.5925531282199274


# Problem 2

### a)

For at-the-money vanilla call， the black scholes delta is 

\begin{align}
\Delta = N(d1)
\end{align}
\begin{align}
d_1 &= \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{align}

If x is close to 0, we can apply Taylor expansion to first order:

\begin{equation}
N(x) \approx N(0) + N'(0) \cdot x.
\end{equation}

N(0) = 0.5, $N'(0) = \frac{1}{\sqrt{2 \pi}}$, and x = d_1

so:
\begin{align}
\Delta &\approx 0.5 + \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma \sqrt{2 \pi}}\\
&\approx 0.5 + 0.4 \cdot \frac{(r + \frac{\sigma^2}{2}) \sqrt{T}}{\sigma}.
\end{align}







plug in numbers that σ = 0.2 and T = 0.25 and r = 0.01.

\begin{align}
\Delta &\approx 0.5 + 0.4 \cdot 0.075\\
&\approx 0.53.
\end{align}

### b)

Second order Taylor Expansion:
    
\begin{equation}
C(x-4) \approx C(4) + C'(4) \cdot (x-4) + 0.5 \cdot C''(4) \cdot (x-4)^2.\\
\approx N(0) + \Delta \cdot x + 0.5 \cdot \Gamma \cdot x^2
\end{equation}

Then, we need to change dollar delta and dollar gamma into delta and gamma

\begin{align}
DollarDelta = 3 
&\implies \Delta = \frac{3}{4}\\
DollarGamma = 0.02
&\implies \Gamma = \frac{1}{8}\\
\end{align}



Let's plug it in:

- Now we have: x = 3.6, Delta = 3/4, Gamma = 1/8

\begin{align}
C(3.6) &\approx 5 + \frac{3}{4} (3.6 - 4) + \frac{1}{16} (3.6 - 4)^2\\
&\approx 5 + \frac{3}{4} (-0.4) + \frac{1}{16} (-0.4)^2\\
&\approx 4.71.
\end{align}